<a href="https://colab.research.google.com/github/XiuqiXi/colabNotebook/blob/main/ManimExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

In [4]:
from manim import *

In [10]:
%%manim -qm -v WARNING SquareToCircle

class SquareToCircle(Scene):
   def construct(self):
      square = Square()
      circle = Circle()
      circle.set_fill(RED, opacity=0.9)
      self.play(Create(circle))
      self.play(Transform(circle, square))
      self.wait()

In [17]:
%%manim -qm -v WARNING SquareAndCircle

class SquareAndCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(PINK, opacity=0.5)  # set the color and transparency

        square = Square()  # create a square
        square.set_fill(BLUE, opacity=0.5)  # set the color and transparency

        square.next_to(circle, UP, buff=-1)  # set the position
        self.play(Create(circle), Create(square))  # show the shapes on screen

In [34]:
%%manim -qm -v WARNING MathTeXDemo

class MathTeXDemo(Scene):
    def construct(self):
        rtarrow0 = MathTex(r"B_{R}=\frac{(1-\chi) \Delta H Y_{\infty} s-c_{p g}\left(T_{i g}-T_{\infty}\right)}{L+\left(\dot{q}_{f, r}^{\prime \prime}-\dot{q}_{s, r}^{\prime \prime}-\dot{q}_{s, c}^{\prime \prime}\right) / \dot{m}_{\mathrm{f}}^{\prime \prime}}", font_size=40)
        rtarrow1 = Tex(r"$B=\frac{\Delta H Y_{\infty \infty} s-c_{p g}\left(T_{\text {ig }}-T_{\infty}\right)}{L+Q}$", font_size=40)

        rtarrow1.next_to(rtarrow0, UP, buff=1.5)  # set the position

        self.play(Create(rtarrow0), Create(rtarrow1))

        self.play(Transform(rtarrow1, rtarrow0))

        self.wait()
